## Imports

In [1]:
%%capture
# Suppress output

# Whether the notebook is run within Google Colab or not
colab = 'google.colab' in str(get_ipython())

# General imports
import numpy as np
import pandas as pd
import torch
# Install needed dependencies on Colab
if colab:
    !pip install transformers
    !pip install torchmetrics
from transformers import DistilBertModel#, DistilBertTokenizerFast

# Enable GPU acceleration, whenever available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Automatically reimport modules at each execution
%reload_ext autoreload
%autoreload 2

In [2]:
if colab:
    !git clone 'https://github.com/michimichiamo/question-answering'

fatal: destination path 'question-answering' already exists and is not an empty directory.


## Read data

In [3]:
# Execute this only to load the dataset in csv format if not already done
# from read_dataset import read_dataset

# dataset = read_dataset(path='training_set.json', validation_set_perc=20)
# train_df = pd.DataFrame(dataset[0], columns=['id', 'title', 'context_id', 'context', 'question', 'start', 'end'])
# train_df.to_csv('train_df.csv')
# val_df = pd.DataFrame(dataset[1], columns=['id', 'title', 'context_id', 'context', 'question', 'start', 'end'])
# val_df.to_csv('val_df.csv')

In [4]:
directory='./' if not colab else './question-answering/'

train_filename = directory+'data/tokenized/train.npz'
val_filename = directory+'data/tokenized/val.npz'

train_data = np.load(train_filename)
val_data = np.load(val_filename)

In [5]:
train_input_ids = train_data['input_ids'].astype('int32')
train_attention_mask = train_data['attention_mask'].astype('int32')
train_answer_start = train_data['answer_start'].astype('int32')
train_answer_end = train_data['answer_end'].astype('int32')

val_input_ids = val_data['input_ids'].astype('int32')
val_attention_mask = val_data['attention_mask'].astype('int32')
val_answer_start = val_data['answer_start'].astype('int32')
val_answer_end = val_data['answer_end'].astype('int32')

## Network

In [6]:
class QA(torch.nn.Module):

    def __init__(self, hidden_size=768, num_labels=2, dropout_rate=0.5):
        super(QA, self).__init__()
        # Device
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        # Parameters
        self.hidden_size = hidden_size
        self.num_labels = num_labels
        
        # Layers
        #self.tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased-distilled-squad')
        self.transformers = DistilBertModel.from_pretrained('distilbert-base-cased-distilled-squad').to(self.device)
        self.transformers.requires_grad_(False)
        self.dropout = torch.nn.Dropout(dropout_rate)
        #self.extra_linear = torch.nn.Linear(self.hidden_size, self.hidden_size)
        #self.extra_linear_tanh = torch.nn.Tanh()
        self.dense = torch.nn.Linear(self.hidden_size, self.num_labels, device=self.device, dtype=torch.float32)

    def forward(self, inputs):
        # Unpack inputs
        input_ids, attention_mask = inputs
        
        # Put to device
        input_ids = input_ids.to(self.device)
        attention_mask = attention_mask.to(self.device)
        
        # Transformers 
        transformed = self.transformers(input_ids=input_ids, attention_mask=attention_mask)
        # Dropout
        dropped = self.dropout(transformed[0])
        # Obtain logits
        logits = self.dense(dropped) #(None, seq_len, hidden_size)*(hidden_size, 2)=(None, seq_len, 2)
        start_logits, end_logits = logits.split(1, dim=-1)    #(None, seq_len, 1), (None, seq_len, 1)
        start_logits = start_logits.squeeze(-1)  #(None, seq_len)
        end_logits = end_logits.squeeze(-1)    #(None, seq_len)
        # --- 4) Prepare output tuple
        outputs = (start_logits, end_logits)
        
        return outputs

In [7]:
class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, input_ids, attention_masks, answer_starts, answer_ends):
        'Initialization'
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.answer_starts = answer_starts
        self.answer_ends = answer_ends

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.input_ids)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        input_id = self.input_ids[index]
        attention_mask = self.attention_masks[index]
        answer_start = self.answer_starts[index]
        answer_end = self.answer_ends[index]

        # Pack input and output
        X = (input_id, attention_mask)
        y = (answer_start, answer_end)

        return X, y

In [8]:
#@title Hyperparameters
batch_size = 256 #@param ["32", "64", "128"] {type:"raw"}
learning_rate = 0.001 #@param ["0.00001", "0.0001", "0.001", "0.01", "0.1", "1"] {type:"raw"}
epochs = 5 #@param {type:"slider", min:5, max:200, step:5}


In [9]:
train_dataset = Dataset(train_input_ids, train_attention_mask, train_answer_start, train_answer_end)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)#, num_workers=2, pin_memory=True)

In [10]:
val_dataset = Dataset(val_input_ids, val_attention_mask, val_answer_start, val_answer_end)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, )#, num_workers=2, pin_memory=True)
val_dataloader = iter(val_dataloader)

In [11]:
import gc

del train_input_ids, train_attention_mask, train_answer_start, train_answer_end
del val_input_ids, val_attention_mask, val_answer_start, val_answer_end

gc.collect()

437

In [12]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

# Create model
net = QA()
net.to(net.device)
optimizer = Adam(net.parameters(), lr=learning_rate, weight_decay=0.01)
loss_fn = CrossEntropyLoss()
n_iter = len(train_dataloader)

In [13]:
from torchmetrics import AveragePrecision, F1

# Define scores and send to device
f1_score = F1(num_classes=net.transformers.config.max_position_embeddings, mdmc_average='global')
f1_score = f1_score.to(device)
average_precision = AveragePrecision(pos_label=1, num_classes=net.transformers.config.max_position_embeddings)
average_precision = average_precision.to(device)

def evaluate(model, inputs, targets):
    # Set evaluation mode
    model.eval()
    # Obtain predictions
    start_preds, end_preds = model.forward(inputs)
    # Unpack targets and send to device
    start_logits, end_logits = targets
    start_logits = start_logits.to(model.device)
    end_logits = end_logits.to(model.device)
    
    # Extract IntTensors for predictions
    start_out, end_out = torch.zeros_like(start_preds, dtype=torch.int16), torch.zeros_like(end_preds, dtype=torch.int16)
    start_out[torch.tensor(range(start_preds.size()[0])), torch.argmax(start_preds, axis=1)] = 1
    end_out[torch.tensor(range(end_preds.size()[0])), torch.argmax(end_preds, axis=1)] = 1

    # Send predictions to device
    start_out.to(model.device)
    end_out.to(model.device)

    # Get F1 scores
    f1_start = f1_score(start_out, start_logits)
    f1_end = f1_score(end_out, end_logits)
    f1 = f1_start + f1_end
    
    # Get Average Precision scores
    avg_start = average_precision(start_out, torch.argmax(start_logits, axis=1))
    avg_end = average_precision(end_out, torch.argmax(end_logits, axis=1))
    avg = avg_start + avg_end

    print(f'f1 score: {f1:.10f}')
    print(f'average precision: {avg:.5f}')
    return f1.to('cpu'), avg.to('cpu')


/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [16]:
loss_history = []
f1_history = []
avg_prec_history = []

for epoch in range(epochs):
    net.train()
    for iteration, (train_inputs, train_targets) in enumerate(train_dataloader):
        net.train()
        # Unpack targets and cast to float
        start_logits, end_logits = train_targets
        start_logits, end_logits = torch.tensor(start_logits, dtype=torch.float32, device=device), torch.tensor(end_logits, dtype=torch.float32, device=device)
        # Forward pass
        optimizer.zero_grad()
        start_out, end_out = net.forward(train_inputs)
        # Loss function
        ## TOCHECK
        loss = loss_fn(start_out, start_logits) + loss_fn(end_out, end_logits)
        # Gradient update
        loss.backward()
        optimizer.step()
#
        # Track loss
        print(f'iteration {iteration+1}/{n_iter}')
        print(f'loss = {loss}')
        loss_history.append(loss)
    
    #if epoch % 5 == 0:
    net.eval()
    val_inputs, val_targets = next(val_dataloader)
    f1, avg_prec = evaluate(net, val_inputs, val_targets)
    f1_history.append(f1)
    avg_prec_history.append(avg_prec)




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


iteration 1/279
loss = 4.62442684173584
iteration 2/279
loss = 5.791830062866211
iteration 3/279
loss = 4.785766124725342
iteration 4/279
loss = 5.347018718719482
iteration 5/279
loss = 5.220499038696289
iteration 6/279
loss = 5.5930986404418945
iteration 7/279
loss = 4.0742878913879395
iteration 8/279
loss = 5.150293350219727
iteration 9/279
loss = 5.935805320739746
iteration 10/279
loss = 6.723872184753418
iteration 11/279
loss = 4.715449333190918
iteration 12/279
loss = 5.010364532470703
iteration 13/279
loss = 6.722583770751953
iteration 14/279
loss = 6.430431842803955
iteration 15/279
loss = 6.160817623138428
iteration 16/279
loss = 3.97371244430542
iteration 17/279
loss = 4.640641689300537
iteration 18/279
loss = 4.438995361328125
iteration 19/279
loss = 4.902169704437256
iteration 20/279
loss = 7.108757972717285
iteration 21/279
loss = 7.52585506439209
iteration 22/279
loss = 5.481133460998535
iteration 23/279
loss = 5.4608659744262695
iteration 24/279
loss = 7.400188446044922
i

/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Argument `pos_label` should be `None` when running multiclass precision recall curve. Got 1
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/torchmetrics/functional/classification/average_precision.py:168: UserWarning: Average precision score for one or more classes was `nan`. Ignoring these classes in average
  UserWarning,


f1 score: 1.9963531494
average precision: 0.81714
iteration 1/279
loss = 4.759305953979492
iteration 2/279
loss = 6.479163646697998
iteration 3/279
loss = 5.152984142303467
iteration 4/279
loss = 5.579827308654785
iteration 5/279
loss = 5.713690280914307
iteration 6/279
loss = 6.076764106750488
iteration 7/279
loss = 4.0215864181518555
iteration 8/279
loss = 5.831201553344727
iteration 9/279
loss = 5.874209403991699
iteration 10/279
loss = 7.099842548370361
iteration 11/279
loss = 4.767147064208984
iteration 12/279
loss = 4.958802223205566
iteration 13/279
loss = 6.635282516479492
iteration 14/279
loss = 6.690831184387207
iteration 15/279
loss = 6.444301605224609
iteration 16/279
loss = 3.9566280841827393
iteration 17/279
loss = 4.724586009979248
iteration 18/279
loss = 4.463770866394043
iteration 19/279
loss = 5.2642645835876465
iteration 20/279
loss = 7.166962623596191
iteration 21/279
loss = 7.462397575378418
iteration 22/279
loss = 5.762126922607422
iteration 23/279
loss = 5.792386

KeyboardInterrupt: ignored